In [58]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import Counter
import theano
theano.config.device = 'gpu'
theano.config.floatX = 'float32'

In [53]:
lb = preprocessing.LabelBinarizer()
data = np.load('out/new_input.npz')
train_x, train_y = data['train'][:,:-1], data['train'][:,-1]
test_x, test_y = data['test'][:,:-1],  data['test'][:,-1]
train_y = np.array([1 if i == 2 else 0 for i in train_y])
test_y = np.array([1 if i == 2 else 0 for i in test_y])
# train_y_1hot = lb.fit(train_y).transform(train_y)
# test_y_1hot = lb.fit(train_y).transform(test_y)
# train_x.shape, train_y.shape, test_x.shape, test_y.shape
Counter(train_y), Counter(test_y)

(Counter({0: 58630, 1: 67343}), Counter({0: 12833, 1: 9711}))

In [60]:
model = Sequential()

model.add(Dense(500, input_dim=200, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dense(200, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dense(50, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dense(1, init='uniform'))
model.add(Activation('sigmoid'))
sgd = SGD(lr=0.06, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',optimizer=sgd)

In [61]:
model.fit(train_x, train_y,nb_epoch=30,batch_size=200)

Epoch 1/30
125973/125973 [==============================] - 60s - loss: 0.5432    
Epoch 2/30
125973/125973 [==============================] - 59s - loss: 0.1031    
Epoch 3/30
125973/125973 [==============================] - 58s - loss: 0.0887    
Epoch 4/30
125973/125973 [==============================] - 59s - loss: 0.0791    
Epoch 5/30
125973/125973 [==============================] - 59s - loss: 0.0647    
Epoch 6/30
125973/125973 [==============================] - 60s - loss: 0.0519    
Epoch 7/30
125973/125973 [==============================] - 60s - loss: 0.0447    
Epoch 8/30
125973/125973 [==============================] - 63s - loss: 0.0400    
Epoch 9/30
125973/125973 [==============================] - 59s - loss: 0.0372    
Epoch 10/30
125973/125973 [==============================] - 62s - loss: 0.0344    
Epoch 11/30
125973/125973 [==============================] - 60s - loss: 0.0320    
Epoch 12/30
125973/125973 [==============================] - 60s - loss: 0.0296    
E

In [63]:
p = model.predict_classes(test_x)
print(accuracy_score(test_y, p))
print(confusion_matrix(test_y, p))

22544/22544 [==============================] - 4s     
0.790809084457
[[8852 3981]
 [ 735 8976]]


In [67]:
#saving model
import json
with open('out/model.txt', 'w') as outfile:
    json.dump(model.to_json(), outfile)